In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* @file NLP基礎/NLP_part4.ipynb
  * @brief NLP基礎 模型實作 

  * 此份程式碼是以教學為目的，附有完整的架構解說。

  * @author 人工智慧科技基金會 AI 工程師 - 康文瑋
  * Email: run963741@aif.tw
  * Resume: https://www.cakeresume.com/run963741

  * 最後更新日期: 2020/11/26

# 過去與展望

傳統的自然語言處理都是透過統計方法將一個句子用各式各樣的統計量表示，例如詞頻、BOW、tf-idf 等等，但是這些統計量都沒有辦法真正表示詞的意義，下圖分別為三個自然語言處理領域常見的應用，這些任務都是傳統基於統計方法難以解決的問題，對於一詞多義這種情況，就無法用簡單統計量來表示。

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1Ft8TZPXM_JjB9Jkaj61gzzAvaeaoN-2f' width="500"/>
<figcaption>Parser, word Sense Disambiguation and coreference resolution</figcaption></center>
</figure>


# 載入套件以及資料集

In [ ]:
import pickle
from gensim.models import word2vec
import random
import logging
import tqdm
import os

os.chdir('/content/drive/Shared drives/類技術班教材/標準版/NLP基礎')

In [ ]:
with open('Data/htl_cutted.pickle', 'rb') as file:
    data = pickle.load(file)

In [ ]:
data[0]

['距離',
 '川沙',
 '公路',
 '較',
 '近',
 ',',
 '但是',
 '公交',
 '指示',
 '不',
 '對',
 ',',
 '如果',
 '是',
 '"',
 '蔡陸線',
 '"',
 '的話',
 ',',
 '會',
 '非常',
 '麻煩',
 '.',
 '建議',
 '用',
 '別的',
 '路線',
 '.',
 '房間',
 '較',
 '爲',
 '簡單',
 '.']

# Label encoding

近年來有許多研究都聚焦在如何將詞用另一種數學形式來表示，第一種作法是 label encoding，簡單來說就是每個詞用一個數字來表示。

這種定義方式最大的缺點在於大部分的詞之間本身是沒有大小關係的，例如下面的程式碼將 Taiwan 定義為 `2`，將 Australia  定義為 `0`，但這樣會導致 2 > 0 $\Rightarrow$ Taiwan > Australia，但是國家本身是沒有大小之分的。

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
country = ['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']

encode = labelencoder.fit_transform(country)

In [ ]:
print('Input: \n', country)
print('Labelencoder: \n', encode)

Input: 
 ['Taiwan', 'Australia', 'Ireland', 'Australia', 'Ireland', 'Taiwan']
Labelencoder: 
 [2 0 1 0 1 2]


# One-hot encoding

另一種方式是 One-hot encoding，也就是將每個詞用一個向量來表示，除了自己位置的數值是 1 之外，其餘都是 0，以下圖為例，顏色總共有 3 種，透過 one-hot encoding 轉換之後，每筆資料就會轉換成三維向量，以第一筆資料 Red 為例，就會轉換成 `[1,0,0]` 的向量，第一個位置為 1 表示紅色，其餘顏色的位置就為 0。

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1U8aE8cVobe5M_N3aAtYJyf_6b_XiS-My' width="500"/>
<figcaption>One-hot encoding</figcaption></center>
</figure>

Resource: https://www.r-bloggers.com/2019/07/how-to-use-recipes-package-from-tidymodels-for-one-hot-encoding-%F0%9F%9B%A0-2/

One-hot encoding 在特徵工程常常使用到，例如在 Dataframe 中常常看見文字需要轉換成 one-hot 再丟給模型或是在分類模型時常常使用在標籤轉換上。

One-hot encoding 有兩個致命的問題，分別是線性獨立 (linear independent) 以及維度災難 (curse of dimensionality)：

* 線性獨立 (linear independent):

以上圖的例子來說，Red 和 Yellow 分別會用 `[1,0,0]` 和 `[0,1,0]` 來表示，若將這兩個向量使用內積相乘會得到 0，在數學上就表示這兩個向量彼此線性獨立，但是在生活中這兩種顏色本身是有一定的相關性的，所以 one-hot encoding 這種轉換方式對於文字來說是不夠好的。

* 維度災難 (curse of dimensionality):

細心的同學應該有發現，透過 one-hot 轉換後，向量的長度會跟字典的大小一樣 (以上圖的例子就是 3)，當字典大小會跟向量長度相同。以中文文本來說，字典大小約在 50,000 左右，這表示每個詞都會用 50,000 維的向量來表示，過高維度的資料會導致模型訓練效率急速下降，而且記憶體大小也有限制，例如以字典大小為 50,000，在記憶體就必須有位置讓你放 $50,000\times 50,000$ 維矩陣，而且大部分的位置都是 0 (sparse matrix)，這樣很沒有效率。

In [ ]:
from sklearn.preprocessing import OneHotEncoder

country_idx = [['Taiwan'],['Australia'],['Ireland'],['Japan']]

enc = OneHotEncoder()
onehot = enc.fit_transform(country_idx).toarray()

In [ ]:
print('Input: \n', country_idx)
print('One-hot encoding: \n', onehot)

Input: 
 [['Taiwan'], ['Australia'], ['Ireland'], ['Japan']]
One-hot encoding: 
 [[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


# Word2vec

word2vec 的概念是於 2013 年在 [Distributed Representations of Words and Phrases
and their Compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) 所提出，也是將每個字表示為一個向量，但是這個向量是用神經網路來訓練的，以下圖為例，將 man 用一個 7 維向量來表示，然後通過模型訓練後，再將 7 維向量降維然後畫在圖上，會形成類似下圖右邊坐標系的效果，從第一個坐標系來看，模型可以學習到男性跟女性之間的關係，也就是：

$$
king - queen=man - woman
$$

在第二個坐標系中，模型學習到了現在進行式與過去式之間的對應關係：

$$
walked - walking = swam - swimming
$$


<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=10meJYPIlusaY4dm_Snfa_fFbxjq7vxSp' width="800"/>
<figcaption>One-hot encoding</figcaption></center>
</figure>

到目前為止，Word2vec 詞向量的概念基本上已經稱霸自然語言處理領域，許多知名演算法都是以詞向量作為模型輸入，那這些向量是怎麼被訓練出來的呢 ? 這邊介紹兩個知名的演算法，分別是 CBOW (Continuous bag of words) 以及 Skip-gram。

## CBOW (Continuous bag of words)

CBOW 的做法類似克漏字測驗，也就是用上下文來預測中間的詞，若模型有能力預測出中間的詞，就表示模型有學習到詞之間的關係：

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1CjMp04zT1Jlh_iK3sMVCydkShCPD0FH2' width="500"/>
<figcaption>CBOW</figcaption></center>
</figure>


第一步：首先將每個詞轉成 one-hot encoding，然後上下文共 $C$ 個 one-hot 做總和，然後再通過矩陣 $W
in R^{V\times N}$，接著再平均，得到 hidden representation $h$，這裡的權重 $W$ 就是由所有詞向量所構成的矩陣，也就是我們等一下要觀察的東西：
$$
h=\frac{1}{C}W\cdot(\sum_{i=1}^Cx_i)
$$


第二步：將 $h$ 乘以權重 $W'\in R^{N\times V}$，$V$ 指的是字典大小：

$$
Y=W'^T\cdot h,\;Y\in R^V
$$


第三步：將預測值 $Y$ 通過 softmax 得到每個詞的概率值，接著就能夠計算 loss 並
更新權重：

$$
softmax(Y)=\frac{\exp(Y_i)}{\sum_{k=1}^V\exp(Y_k)}
$$

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1yCedxszktzJwLbVvwo5fgJKYD3lLy3Ct' width="500"/>
<figcaption>CBOW</figcaption></center>
</figure>


In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html
# sg=0 CBOW ; sg=1 skip-gram
# size: 詞向量維度
# min_count: 頻率大於等於 3 才會作為 word2vec 的字典使用
# window: cbow 模型底下一次取多少詞來預測中間的詞
bow_model = word2vec.Word2Vec(size=128, min_count=3, window=5, sg=0)

In [ ]:
# 首先建立字典
bow_model.build_vocab(data)

In [ ]:
# train word2vec model ; shuffle data every epoch
for i in tqdm.tqdm(range(20)):
    random.shuffle(data)
    bow_model.train(data, total_examples=len(data), epochs=2)

bow_model.save("word2vec_model/cbow.model")

100%|██████████| 20/20 [00:18<00:00,  1.07it/s]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# 讀取模型
bow_model = word2vec.Word2Vec.load("word2vec_model/cbow.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# 觀察 飯店 的詞向量
bow_model.wv['飯店']

array([-1.0701672 , -0.5366705 , -0.14141004,  1.4369427 , -0.7404118 ,
        0.5987442 ,  1.1751707 ,  0.49842593,  1.5675281 , -0.02220872,
        1.1056122 ,  0.61487997, -1.7171193 , -1.9098822 , -0.25944138,
        1.1256552 , -0.16464075,  0.5829185 , -1.4798691 , -0.44180414,
       -0.02606585,  1.9827013 ,  0.15339974,  0.38701537, -0.11875502,
       -0.62823737, -0.9901219 , -1.8006724 ,  2.3847196 , -0.66525346,
       -0.64452624, -2.0062046 ,  1.1393052 ,  3.2320094 , -0.1776061 ,
       -1.0081499 , -0.36308467, -2.11115   , -0.09659322, -0.14233512,
        1.1370134 ,  0.08736645, -2.405578  ,  0.9725603 , -1.3069246 ,
       -0.48618224,  0.6675372 ,  0.1661223 , -0.62522864, -0.68891925,
        0.6024552 ,  1.5583718 ,  0.15468223, -0.4883932 , -0.26298532,
       -1.2618384 , -3.9300814 , -0.52221674, -2.6756866 ,  1.3235472 ,
       -2.1499412 , -0.9530751 , -0.6591422 ,  1.8895234 ,  0.40236062,
       -0.83020693,  1.0518736 ,  0.83717227,  0.477712  , -0.52

In [ ]:
# 看看與 飯店 和字典中最接近的詞是什麼
bow_model.wv.most_similar('飯店')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('賓館', 0.6172376275062561),
 ('酒店', 0.5052703619003296),
 ('城市', 0.4633227586746216),
 ('南昌', 0.36634230613708496),
 ('商店', 0.3613012433052063),
 ('地區', 0.3563742935657501),
 ('早茶', 0.35457515716552734),
 ('小吃店', 0.3527277708053589),
 ('居民區', 0.34699302911758423),
 ('旅館', 0.34555113315582275)]

In [ ]:
# 找相對應的詞，冬天 - 暖氣 + 夏天 = ?
bow_model.wv.most_similar(positive=['冬天', '暖氣'], negative=['夏天'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('製', 0.6001927256584167),
 ('空調', 0.5621954202651978),
 ('冷氣', 0.5377000570297241),
 ('暖風', 0.5338841676712036),
 ('水溫', 0.5200881958007812),
 ('溫度', 0.49906039237976074),
 ('熱', 0.4924931526184082),
 ('熱水', 0.48980918526649475),
 ('調節', 0.4820823669433594),
 ('開關', 0.4768384099006653)]

## Skip-gram 

Skip-gram 的預測目標與 CBOW 相反，是使用某一詞去預測上下文，以下圖為例，我們使用中間的詞 `很大, 記得帶一支`，來預測上下文，透過這種方式來訓練詞向量：




<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=15CCMlDIT41AvmcVki73ut4rG3EMncljv' width="700"/>
<figcaption>Skip-gram</figcaption></center>
</figure>

在開始訓練前，skip-gram 會假設一個 window_size，以下圖為例，window_size 為3，並使用中間的詞 `passes` 來預測上下文 `who` 以及 `the`。

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1ZMQ0m62jrYiIAxG8W0HE4HaUSexnOeAO' width="700"/>
<figcaption>Skip-gram with window size equal to three</figcaption></center>
</figure>

Resource: https://aegis4048.github.io/demystifying_neural_network_in_skip_gram_language_modeling


假設 window size 為 5，則所有情況如下圖，也就是用 **Target word** 預測 **Context** 裡面的詞，以第一列 (row) 為例，就會產生 (the,man), (the,who) 這兩筆訓練樣本，也就是分別用 `the` 來預測 `man`，然後再用 `the` 來預測 `who`。

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1-3DEDTyB4YnG7yDMaAyzesWvmUCOHL89' width="700"/>
<figcaption>Skip-gram with window size equal to five</figcaption></center>
</figure>

Resource: https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html#context-based-skip-gram-model

Skip-gram 運作流程如下圖：

第一步：將 `target` 轉換為 One-hot encoding $X\in R^V$ ($V$ 為字典大小)，然後再乘以權重 $W\in R^{V\times N}$ ($N$ 為詞向量大小)，得到 hidden representation $h\in R^N$：

$$
h=XW
$$


第二步：將 hidden representation $h$ 乘以權重 $W'\in R^{N\times V}$，得到預測值 $Y\in R^V$：

$$
Y=hW'
$$


第三步：將預測值 $Y$ 通過 softmax 得到每個詞的概率值，接著就能夠計算 loss 並
更新權重：

$$
softmax(Y)=\frac{\exp(Y_i)}{\sum_{k=1}^V\exp(Y_k)}
$$

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=11YZ_IpprytY4ybGt85f7CabBDbUtKvug' width="700"/>
<figcaption>Skip-gram pipeline</figcaption></center>
</figure>

In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html
# sg=0 CBOW ; sg=1 skip-gram
# size: 詞向量維度
# min_count: 頻率大於等於 3 才會作為 word2vec 的字典使用
# window: cbow 模型底下一次取多少詞來預測中間的詞
skipgram_model = word2vec.Word2Vec(size=128, min_count=3, window=3, sg=1)

In [ ]:
# 首先建立字典
skipgram_model.build_vocab(data)

In [ ]:
# train word2vec model ; shuffle data every epoch
for i in tqdm.tqdm(range(20)):
    random.shuffle(data)
    skipgram_model.train(data, total_examples=len(data), epochs=2)

skipgram_model.save("word2vec_model/skipgram.model")

100%|██████████| 20/20 [00:42<00:00,  2.13s/it]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# 讀取模型
skipgram_model = word2vec.Word2Vec.load("word2vec_model/skipgram.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# 觀察 飯店 的詞向量
skipgram_model.wv['飯店']

array([-0.1921961 ,  0.6429113 ,  0.84826696, -0.17512208, -0.16462117,
       -0.18926574,  0.20201826, -0.21031773,  0.45067227,  0.03488045,
        0.29420573,  0.2214367 , -0.01396547, -0.68384683, -0.3797484 ,
       -0.07619353,  0.31956688,  0.12331374, -0.7047422 , -0.1487584 ,
        0.43465686,  0.2951043 ,  0.07627244,  0.252892  , -0.0542222 ,
       -0.3137878 , -0.23479237, -0.9533322 ,  0.5642962 ,  0.17222688,
        0.08840205, -0.28939658,  0.37507924, -0.01490949, -0.37481546,
       -0.71201515, -0.11033753, -0.4433349 ,  0.284574  ,  0.2573987 ,
        0.49753553, -0.4114739 , -0.57567847, -0.363803  , -0.239369  ,
       -0.06081809, -0.29769722,  0.09416001, -0.40276113, -0.41059715,
        0.3371592 ,  0.15318027,  0.189732  , -0.09726161,  0.06078521,
       -0.18523186, -0.791787  , -0.13431887, -0.8530894 , -0.26824167,
        0.10745721,  0.08118854,  0.12035467,  0.29548284,  0.23184474,
        0.10766387, -0.06128077,  0.00350666,  0.14721055,  0.27

In [ ]:
# 看看與 飯店 和字典中最接近的詞是什麼
skipgram_model.wv.most_similar('飯店')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('酒店', 0.5937623381614685),
 ('甜品店', 0.4943004250526428),
 ('賓館', 0.48139411211013794),
 ('旅社', 0.44420185685157776),
 ('唐山', 0.4416014850139618),
 ('西溪', 0.4403097629547119),
 ('揚子島', 0.43499356508255005),
 ('向來', 0.4346087574958801),
 ('王府', 0.42976114153862),
 ('市郊', 0.42503321170806885)]

In [ ]:
# 找相對應的詞，冬天 - 暖氣 + 夏天 = ?
skipgram_model.wv.most_similar(positive=['冬天', '暖氣'], negative=['夏天'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('空調', 0.4979294240474701),
 ('製', 0.48356133699417114),
 ('褥子', 0.48000916838645935),
 ('tnnd', 0.4640786647796631),
 ('強勁', 0.45863544940948486),
 ('角房', 0.4522526264190674),
 ('充足', 0.4378127455711365),
 ('洗澡間', 0.4367930591106415),
 ('暖風', 0.4349173903465271),
 ('電話機', 0.4324340522289276)]